<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys ; sys.path.insert(0, "../")

In [ ]:
from argo.workflows.sdk import Workflow

from argo.workflows.sdk.tasks import *
from argo.workflows.sdk.templates import *

In [ ]:
import yaml

from pprint import pprint

from argo.workflows.sdk._utils import sanitize_for_serialization

---

In [ ]:
from pathlib import Path

manifest = Path("./dag-diamond.yaml").read_text()
print(manifest)

In [ ]:
class DagDiamond(Workflow):
    
    @task
    @arguments.parameter(name="message", value="A")
    def A(self, message: V1alpha1Parameter) -> V1alpha1Template:
        return self.echo(message=message)
    
    @task
    @arguments.parameter(name="message", value="B")
    @dependencies(["A"])
    def B(self, message: V1alpha1Parameter) -> V1alpha1Template:
        return self.echo(message=message)
    
    @task
    @arguments.parameter(name="message", value="C")
    @dependencies(["A"])
    def C(self, message: V1alpha1Parameter) -> V1alpha1Template:
        return self.echo(message=message)
    
    @task
    @arguments.parameter(name="message", value="D")
    @dependencies(["B", "C"])
    def D(self, message: V1alpha1Parameter) -> V1alpha1Template:
        return self.echo(message=message)
    
    @template
    @inputs.parameter(name="message")
    def echo(self, message: V1alpha1Parameter) -> V1Container:
        container = V1Container(
            image="alpine:3.7",
            name="echo",
            command=["echo", "{{inputs.parameters.message}}"],
        )
        
        return container

wf = DagDiamond()
wf

---

In [ ]:
pprint(sanitize_for_serialization(wf))

In [ ]:
pprint(yaml.safe_load(manifest))

In [ ]:
assert sanitize_for_serialization(wf) == yaml.safe_load(manifest), "Manifests don't match."